<a href="https://colab.research.google.com/github/hvasquez28/g1-ds_santander-f2-team7-Phyton/blob/Harold/Entrega_Final_Procesamiento_DanielTapia_LuisZanabria_NorbertoPena_HaroldVasquez/Top20MercadolibreMx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Encontrar los top 20 productos más vendidos en MercadoLibre México

Se quiere buscar los top 20 productos más vendidos en la API de MercadoLibre, específicamente en la página de México. Para esto, se utilizó la documentación de la siguiente página web: https://developers.mercadolibre.cl/es_ar/mas-vendidos-en-mercado-libre.



### 1) Registro de la APP

Vamos a implementar un programa que realice una llamada HTTP a la API de NASA.

Puedes leer la documentación de la API [aquí](https://api.nasa.gov/), bajo el título de "Asteroids NeoWs".

### 2) Obtención del token

se uso esta pagina https://developers.mercadolibre.com.ar/es_ar/autenticacion-y-autorizacion

In [ ]:
## Imports necesarios
import requests
import pandas as pd

In [ ]:
token={ 'Authorization': 'Bearer <your_token>' }


### 3) Identificacion del site

Cada una de las siguientes llamadas estan agrupadas por sites (paises).
en la siquiente página web hay detalles de los sites cubiertos por ML https://developers.mercadolibre.cl/es_ar/categorias-y-publicaciones

Por lo tanto, primero debemos consultar con todos ellos y buscar el que nos interesa: México.

In [ ]:
endpoint = 'https://api.mercadolibre.com/sites'
r = requests.get(endpoint, headers=token)
r.status_code

In [ ]:
# si en la respuesta anterior obtenemos un 200, porcedemos a crear un DF con la respeusta obtenida

json = r.json()
sites =  pd.json_normalize(json['id','name'])
sites

In [ ]:
# Ahora buscamos el ID del site que nos interesa
sites.loc[sites['name'] == 'Mexico', ['id']]
# site_id

### 4) Obtención del id de Categoria

Dado que para hacer la consulta que requerimos, se necesita un id de la categoria de interes, debemos primero indentificar dicho id. Para ello, haremos una llamada al EndPoint descrito en esta página https://developers.mercadolibre.cl/categoriza-productos

In [ ]:
endpoint = 'https://api.mercadolibre.com/sites/'+site_id+'/categories'
r = requests.get(endpoint, headers=token)
r.status_code

In [ ]:
# si en la respuesta anterior obtenemos un 200, procedemos a crear un DF con la respuesta obtenida

json = r.json()
categories =  pd.json_normalize(json['id','name'])
categories

In [ ]:
# Revisando la respuesta obtenida, se deduce que la categoria que nos interesa es 'Deportes y Fitness'. Por eso buscamos el Id de esta
sites.loc[sites['name'] == 'Deportes y Fitness', ['id']]
# category_id

### 5) Obtención de los top 20 productos más vendidos en 'Deportes y Fitness' en MercadoLibre México

Finalmente, podremos obtener lo que nos interesa para este Playbook, los 20 productos más vendidos en Mexico al momento de esta consulta.

Fecha y hora de la cosnsulta: 9/17/2022 a las 4:00 pm CST

In [ ]:
endpoint = 'https://api.mercadolibre.com/highlights/'+site_id+'/category/'+category_id
r = requests.get(endpoint, headers=token)
r.status_code

In [ ]:
# si en la respuesta anterior obtenemos un 200, procedemos a crear un DF con la respuesta obtenida

json = r.json()
top20 =  pd.json_normalize(json['content'])
top20

In [ ]:
# Para conocer los detalles de cada item, automatizamos la llamada al siguiente EndPoint (https://developers.mercadolibre.cl/buscador-de-productos)
# con cada uno de ellos y creamos un data frame compleot con todos ellos



dict_datos = {}

for i in range(0, 20):
    category_id = top20.loc[sites['position'] == i+1, ['id']]
    endpoint = 'https://api.mercadolibre.com/products/' + category_id

    try:
        time.sleep(5)
        r = requests.get(endpoint, headers=token, timeout=5)

        if r.status_code == 200:
            json = r.json()

            data = json[]
            dict_datos[i] = data
    except:
        continue

for key in dict_datos:
    normalized = pd.json_normalize(dict_datos[key])
    df = pd.DataFrame.from_dict(normalized)
    dict_datos[key] = df

lista_de_dataframes = []

for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

df_completo = pd.concat(lista_de_dataframes, axis=0).reset_index(drop=True)
df_completo